This is a standalone python notebook file where the highway environment is dicretized to a custom level of coarse granularity to reduce simulation complexity for faster generation of MDP tables.

In [1]:
# Move up one directory level to import local instance of highway environment
%cd ..
# %pwd

d:\WorkFiles\ProjectCode\ActiveProjects\HoLab\Mine\HighwayEnv-TRI


Import necessary modules

In [2]:
from highway_env.envs.discreteMDP_wrapper import HighwayDiscreteMDP, OptimalPolicy

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)    # Other options: INFO, WARNING, ERROR, CRITICAL

import numpy as np
import itertools as it

MDP Stuff

In [ ]:
# Define Configuration
num_of_vehicles = 2
num_of_ice = 5
env_length = 3000   # Max car speed (for MDP vehicle) is 30m/s At constant max speed, the simulation will last 100s in simulation time
lane_count = 3
config = {    
        ## Parameters of interest ##
        "observation": {
            # For more details about observation parameters check out "highway_env\envs\common\observation.py"
            "type": "Kinematics",
            "vehicles_count": num_of_vehicles+num_of_ice+5,   # Number of vehicles (and objects) to show in the observation. 
                                                                #   Keep greater than value of vehicles out outside obervation
                                                                #   dictionary to observe all vehicles in the environment.
            "features": ["presence", "x", "y", "vx", "vy"],# "heading"],
            "normalize": False, # Normalize object coordinates
            "absolute": True,   # Provide absolute coordinate of vehicles
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True,
            "see_behind": True  # Report vehicles behind the ego vehicle
            },
        ## Parameters specialized for the icy highway environment ##
        "ice_count": num_of_ice,    # Number of ice sheets in the environment
        "env_len":  env_length,    # Length of the road
        ## Keep these to default, because the fast versions of the environments implement different values ##
        ## of these variables for faster execution ##
        "vehicles_count": num_of_vehicles,
        "lanes_count": lane_count,
        "simulation_frequency": 5,
        "duration": (env_length/20)+5,  # [in simulation seconds], minimum speed for MDP vehicle is 20m/s, with extra 5s
        "disable_collision_checks": True,    # Check collisions for other vehicles
        "enable_lane_change": False,
        ## Other parameters aleady set by default configurations ##
        # "action": {
        #     "type": "DiscreteMetaAction",
        # },
        # "controlled_vehicles": 1,
        # "initial_lane_id": None,
        # "ego_spacing": 2,
        # "vehicles_density": 1,
        # "collision_reward": -1,  # The reward received when colliding with a vehicle.
        # "right_lane_reward": 0.1,  # The reward received when driving on the right-most lanes, linearly mapped to
        # # zero for other lanes.
        # "high_speed_reward": 0.4,  # The reward received when driving at full speed, linearly mapped to zero for
        # # lower speeds according to config["reward_speed_range"].
        # "lane_change_reward": 0,  # The reward received at each lane change action.
        # "reward_speed_range": [20, 30],
        # "normalize_reward": True,
        # "offroad_terminal": False
        }
# highway_mdp = HighwayDiscreteMDP('highway-v0', config=config, render_mode='rgb_array')
# highway_mdp = HighwayDiscreteMDP('highway-v0', config=config, render_mode=None)
# highway_mdp = HighwayDiscreteMDP('highway-fast-v0', config=config, render_mode=None)
# highway_mdp = HighwayDiscreteMDP('highway-icy-v0', config=config, render_mode=None)
# highway_mdp = HighwayDiscreteMDP('highway-icy-fast-v0', config=config, render_mode=None)
highway_mdp = HighwayDiscreteMDP('highway-icy-custom-v0', config=config, render_mode=None)

In [4]:
# Create discrete world grid
temp = highway_mdp.env.unwrapped.road.network.lanes_list()
x_max = max([lane.end[0] for lane in temp])
y_max = max([lane.end[1] for lane in temp])
#2# |Divide the environment into 600 blocks along the x-axis
x_grid = np.linspace(0, x_max, 600)
y_grid = np.linspace(0, y_max, lane_count)
state_list = list(it.product(x_grid, y_grid))
[x_max, y_max]

[3000.0, 8.0]

In [ ]:
# veh_speed for MDP vehicle can be 20/25/30
temp = highway_mdp.populate_MDPtable_StaticGridworld(state_list, parallel_exec=False, veh_speed = 25)   # notebook does not handle multiprocessing well

Simulation start


In [6]:
temp

MDPTable(start_state=None, transition={((0.0, 0.0), 0): {(33.333333333333336, 0.0): 1}, ((0.0, 0.0), 1): {(33.333333333333336, 4.0): 1}, ((0.0, 0.0), 2): {(33.333333333333336, 8.0): 1}, ((0.0, 0.0), 3): {(33.333333333333336, 4.0): 1}, ((0.0, 0.0), 4): {(33.333333333333336, 4.0): 1}, ((0.0, 4.0), 0): {(33.333333333333336, 0.0): 1}, ((0.0, 4.0), 1): {(33.333333333333336, 4.0): 1}, ((0.0, 4.0), 2): {(33.333333333333336, 8.0): 1}, ((0.0, 4.0), 3): {(33.333333333333336, 4.0): 1}, ((0.0, 4.0), 4): {(33.333333333333336, 4.0): 1}, ((0.0, 8.0), 0): {(33.333333333333336, 0.0): 1}, ((0.0, 8.0), 1): {(33.333333333333336, 4.0): 1}, ((0.0, 8.0), 2): {(33.333333333333336, 8.0): 1}, ((0.0, 8.0), 3): {(33.333333333333336, 4.0): 1}, ((0.0, 8.0), 4): {(33.333333333333336, 4.0): 1}, ((33.333333333333336, 0.0), 0): {(66.66666666666667, 0.0): 1}, ((33.333333333333336, 0.0), 1): {(66.66666666666667, 4.0): 1}, ((33.333333333333336, 0.0), 2): {(66.66666666666667, 8.0): 1}, ((33.333333333333336, 0.0), 3): {(66.

In [11]:
class test:
    def __init__(self):
         self.target_speeds = np.linspace(20, 30, 3)

    def speed_to_index(self, speed: float) -> int:
            """
            Find the index of the closest speed allowed to a given speed.

            Assumes a uniform list of target speeds to avoid searching for the closest target speed

            :param speed: an input speed [m/s]
            :return: the index of the closest speed allowed []
            """
            x = (speed - self.target_speeds[0]) / (
                self.target_speeds[-1] - self.target_speeds[0]
            )
            return np.int64(
                np.clip(
                    np.round(x * (self.target_speeds.size - 1)),
                    0,
                    self.target_speeds.size - 1,
                )
            )

test().speed_to_index(30.0)

2

Construal code

In [ ]:
# Construal MDP Code
construals = highway_mdp.get_construals_singleobj()
# len(construals)

construal_policies = dict()
for constr in construals:
    print("currently running contrual ", len(construal_policies)+1)
    # print(constr.env.unwrapped.road.vehicles)
    # print(constr.env.unwrapped.road.objects)
    # print("----------------")
    constr.populate_MDPtable(max_depth = max_depth)
    value_function = constr.value_iteration()
    policy = OptimalPolicy(constr)
    construal_policies[constr] = policy

In [7]:
construal_rewards = {}
for constr, policy in construal_policies.items():
    actns = policy(constr.current_state)
    print(actns)
    temp_mdp = highway_mdp.get_copy()
    obs, rwrd, done, truncated, info = temp_mdp.step(actns[0])
    construal_rewards[constr] = rwrd

# TODO: take optimal actions based on each policy and calculate reward
# Question: The behavior of other vehicles will also change in these construals 
#           (do we account for that by repreating the construal creation process at each timestep?)

# #2# The below oprations can be repeated upto max depth iterations, to execute all actions calculated by the current policy
# acts = policy(highway_mdp.current_state)
# _ = highway_mdp.step(acts[0])

[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]


c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]


c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]


...

c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]


c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]


c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [6]:
construal_rewards
# len(highway_mdp.env.unwrapped.road.vehicles)
# highway_mdp.env.unwrapped.road.vehicles
# len(highway_mdp.env.unwrapped.road.objects)
# highway_mdp.env.unwrapped.road.objects
# construals[3].env.unwrapped.road.vehicles
# construals[1].env.unwrapped.road.objects

{<highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x22279d3ffe0>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227cd14590>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d6378c0>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227ccb4f20>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d8589e0>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d991ac0>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d8a6e40>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d858800>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d7c7e30>: 0.8666666666666667,
 <highway_env.envs.discreteMDP_wrapper.HighwayDiscreteMDP at 0x2227d9ce0c0>: 0.8666666666666667,
 <highway_env.envs.discreteMDP

In [15]:
# construal_policies[0][0].first_state
# construal_policies[0][0].current_state
# policy(construal_policies[0][0].current_state)

(frozendict.frozendict({'position': (154.56, 4.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (151.09, 4.0), 'speed': (0.0, 0.0), 'heading': -1}),
 frozendict.frozendict({'position': (176.5, 0.0), 'speed': (23.3, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (197.53, 0.0), 'speed': (21.31, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (218.85, 8.0), 'speed': (0.0, 0.0), 'heading': -1}),
 frozendict.frozendict({'position': (221.24, 8.0), 'speed': (23.13, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (242.76, 0.0), 'speed': (22.4, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (265.23, 4.0), 'speed': (22.53, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (268.53, 0.0), 'speed': (0.0, 0.0), 'heading': -1}),
 frozendict.frozendict({'position': (288.73, 8.0), 'speed': (21.33, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (294.44, 0.0), 'speed': (0.0, 0.0), 'heading': -1}),
 fro

TODO:
1. Create a m-vehicle n-icesheet environment
2. Split it into x=m+n+1 separate environments each with a single ice-sheet or vehicle or completely empty
3. Calculate the optimal policy for each of the x environments
4. Run each of those poliies in the oriinal m-vehicle n-icesheet environment.

In [4]:
# Simple MDP Code
highway_mdp_current = highway_mdp.get_copy()

#2# Set up the MDP
_ = highway_mdp_current.populate_MDPtable(max_depth = max_depth)
# matMDP = highway_mdp_current.get_MDPmatrices()    # value iteration already calls this function
value_function = highway_mdp_current.value_iteration()
policy = OptimalPolicy(highway_mdp_current)

#2# The below oprations can be repeated upto max depth iterations, to execute all actions calculated by the current policy
acts = policy(highway_mdp_current.current_state)
_ = highway_mdp_current.step(acts[0])

#2# OR optimal policy can be calculated for each action


INFO:root:Current Depth: 3 | Frontier: 0 | Visited: 28 | Transitions:55
c:\Users\sb10438\AppData\Local\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [8]:
class A:
  @classmethod
  def foo(cls): print('A.foo')
  def bar(self): self.foo()

class B(A):
  @classmethod
  def foo(cls): print('B.foo')

a = A()                
a.bar()
b = B()
b.bar()  

A.foo
B.foo


Print MDP Tables

In [5]:
print(highway_mdp.env.unwrapped.action_type.actions_indexes)

highway_mdp.currMDP.transition

{'LANE_LEFT': 0, 'IDLE': 1, 'LANE_RIGHT': 2, 'FASTER': 3, 'SLOWER': 4}


{(frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  0): {frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  1): {frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  2): {frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  3): {frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  4): {frozendict.frozendict({'position': (176.23, 0.0), 'speed': (20.66, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'pos

In [6]:
highway_mdp.currMDP.reward

{(frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  0,
  frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0})): 0.7999999999999999,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  1,
  frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0})): 0.7999999999999999,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  2,
  frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043})): 0.8327113850086175,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  3,
  frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0})): 0.9157750342935529,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  4,
  frozendict.frozendict({'position': (176.23, 0.0), 'speed

In [7]:
highway_mdp.currMDP.absorption

{frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): False,
 frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (176.23, 0.0), 'speed': (20.66, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (196.57, 0.0), 'speed': (20.09, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (196.19, 3.6), 'speed': (20.05, 1.16), 'heading': 0.058}): False,
 frozendict.frozendict({'position': (198.96, 0.0), 'speed': (24.43, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (203.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (203.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): False,
 frozendict.frozendict({'position': (206.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): False,
 fr

Find optimal policy

# TEST CODE

In [ ]:
import functools
import multiprocessing as mp
process_count = (mp.cpu_count()-2)
from pathos.multiprocessing import ProcessingPool as Pool    # Can run multiprocessing in interactive shell


max_depth = 2   # The number of steps to plan ahead
actions = highway_mdp.actions

start_env = (highway_mdp.initial_state, 0, highway_mdp)
visited = set()
transitions = {}
rewards = {}
absorption = {}
frontier = {start_env}
env2close = set()   # Maintain list of environments to close
while frontier:
    state, depth, curr_MDPstate = frontier.pop()
    visited.add(state)
    env2close.add(curr_MDPstate)
    if depth < max_depth:
        with Pool(process_count) as pool:
            return_vals = pool.map(functools.partial(HighwayDiscreteMDP.exeuteAction, envMDP=curr_MDPstate), actions)
        for action, next_state, trans_prob, reward, done, truncated, info, updated_MDPstate in return_vals:
            logging.debug(str(state[0]) + ' | ' + str(action) + ' | ' + str(next_state[0]))
            #2# Populate transitions table
            if (state[0], action) not in transitions:
                transitions[(state[0], action)] = {}
            if next_state[0] not in transitions[(state[0], action)]:
                transitions[(state[0], action)][next_state[0]] = trans_prob
            #2# Populate reward functions
            if (state[0], action) in rewards:
                rewards[(state[0], action, next_state[0])] += reward * trans_prob
            else:
                rewards[(state[0], action, next_state[0])] = reward * trans_prob
            #2# set absorption states
            absorption[next_state[0]] = done or truncated
            #2# Populate visited
            if next_state not in visited:
                frontier.add((next_state, depth + 1, updated_MDPstate))
            else:
                updated_MDPstate.env.close()
        MDPstatus = "Current Depth: " + str(depth) + " | Frontier: " + str(len(frontier)) +\
                    " | Visited: " + str(len(visited)) + " | Transitions:" + str(len(transitions))
        logging.info(MDPstatus)
env2close.discard(start_env[2]) #Keep the first step active
for curr_MDPstate in env2close:
    # close all duplicate environments
    curr_MDPstate.env.close()
env2close = set()   # Close ununsed environments at the end of the loop
# print(highway_mdp.actions)
highway_mdp.create_MDPTable(transition=transitions, absorption=absorption, reward=rewards, state_list=visited, action_list=actions)

INFO:root:Current Depth: 0 | Frontier: 5 | Visited: 1 | Transitions:5
INFO:root:Current Depth: 1 | Frontier: 9 | Visited: 2 | Transitions:10
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 3 | Transitions:15
INFO:root:Current Depth: 1 | Frontier: 16 | Visited: 5 | Transitions:20
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 12 | Transitions:25
INFO:root:Current Depth: 1 | Frontier: 4 | Visited: 17 | Transitions:25


In [ ]:
# Test differences in environment state for situations with same ego-vehivle state
visited = highway_mdp.populate_MDPtable(max_depth = 4)

for v1 in list(visited)[:-1]:
    for v2 in list(visited)[1:]:
        if v1 != v2 and v1[0] == v2[0]:
            print(v1[0],v2[0])
            for veh1, veh2 in zip(v1,v2):
                if veh1 != veh2:
                    print(veh1, veh2)
            print('\n')


frozendict.frozendict({'position': (251.43, 0.0), 'speed': (25.0, -0.0), 'heading': -0.0}) frozendict.frozendict({'position': (251.43, 0.0), 'speed': (25.0, -0.0), 'heading': -0.0})
frozendict.frozendict({'position': (250.74, 4.0), 'speed': (19.92, 0.01), 'heading': 0.0}) frozendict.frozendict({'position': (248.15, 3.95), 'speed': (19.14, 0.15), 'heading': 0.008})


frozendict.frozendict({'position': (253.83, 0.0), 'speed': (29.34, -0.0), 'heading': -0.0}) frozendict.frozendict({'position': (253.83, 0.0), 'speed': (29.34, -0.0), 'heading': -0.0})
frozendict.frozendict({'position': (248.15, 3.95), 'speed': (19.14, 0.15), 'heading': 0.008}) frozendict.frozendict({'position': (250.74, 4.0), 'speed': (19.92, 0.01), 'heading': 0.0})


frozendict.frozendict({'position': (244.38, 0.0), 'speed': (20.09, -0.0), 'heading': -0.0}) frozendict.frozendict({'position': (244.38, 0.0), 'speed': (20.09, -0.0), 'heading': -0.0})
frozendict.frozendict({'position': (248.15, 3.95), 'speed': (19.14, 0.15), '

In [ ]:
# |DEBUGGING CODE

print(highway_mdp.env.unwrapped.road.vehicles[0])
tmp = highway_mdp.env()
print(tmp.unwrapped.road.vehicles[0])
highway_mdp.step(1)
print(highway_mdp.env.unwrapped.road.vehicles[0])
print(tmp.unwrapped.road.vehicles[0])
highway_mdp.env = tmp
print(highway_mdp.env.unwrapped.road.vehicles[0])


# dir(highway_mdp.env.unwrapped)

next_state, reward, done, truncated, info = highway_mdp.step(1)
# next_state


MDPVehicle #544: [207.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]
1 [1.   1.   0.08 1.   0.   0.  ]
MDPVehicle #544: [232.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]


In [ ]:
config = {
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 50,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "normalize": False,
            "absolute": True,
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True
            }
        }

env = gym.make('highway-v0', config=config, render_mode='human')

In [75]:
vehicles = []
for veh_state in obs:
    feature_vals = {k: v for k,v in zip(config["observation"]["features"], veh_state)}
    veh = {}
    veh["position"] = tuple(np.round((feature_vals["x"],feature_vals["y"]), 2))
    veh["speed"] = tuple(np.round((feature_vals["vx"],feature_vals["vy"]), 2))
    veh["heading"] = np.round(feature_vals["heading"], 3)
    vehicles.append(frozendict(veh))
vehicles = tuple(vehicles)
vehicles

(frozendict.frozendict({'position': (1.0, 0.08), 'speed': (1.0, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.08, 0.0), 'speed': (-0.24, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.33, -0.04), 'speed': (-0.18, 0.0), 'heading': 0.003}),
 frozendict.frozendict({'position': (0.45, 0.04), 'speed': (-0.31, 0.0), 'heading': 0.005}),
 frozendict.frozendict({'position': (0.72, 0.0), 'speed': (-0.19, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.93, 0.04), 'speed': (-0.19, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (1.0, -0.08), 'speed': (-0.27, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (1.0, -0.04), 'speed': (-0.24, -0.0), 'heading': -0.0}),
 frozendict.frozendict({'position': (1.0, -0.0), 'speed': (-0.16, 0.0), 'heading': 0.001}),
 frozendict.frozendict({'position': (1.0, -0.08), 'speed': (-0.28, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.0, 0.0), 'speed': (0.0, 0.0), 'heading': 0.0}),


In [77]:
import sys

# sys.getsizeof(transitions)
sys.getsizeof(highway_mdp.copy_env())
# sys.getsizeof(int(10.0))


48

Profiling HighwayEnv

In [1]:
# |Move up one directory level to import local instance of highway environment
%cd ..
# %pwd

import gymnasium
import highway_env
# from matplotlib import pyplot as plt
# %matplotlib inline

import numpy as np

d:\WorkFiles\ProjectCode\ActiveProjects\HoLab\Mine\HighwayEnv-TRI


In [2]:
# Define Configuration
num_of_vehicles = 2
num_of_ice = 5
env_length = 3000
lane_count = 3
config = {    
        ## Parameters of interest ##
        "observation": {
            # For more details about observation parameters check out "highway_env\envs\common\observation.py"
            "type": "Kinematics",
            "vehicles_count": num_of_vehicles+num_of_ice+5,   # Number of vehicles (and objects) to show in the observation. 
                                                                #   Keep greater than value of vehicles out outside obervation
                                                                #   dictionary to observe all vehicles in the environment.
            "features": ["presence", "x", "y", "vx", "vy"],# "heading"],
            "normalize": False, # Normalize object coordinates
            "absolute": True,   # Provide absolute coordinate of vehicles
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True,
            "see_behind": True  # Report vehicles behind the ego vehicle
            },
        ## Parameters specialized for the icy highway environment ##
        "ice_count": num_of_ice,    # Number of ice sheets in the environment
        "env_len":  env_length,    # Length of the road
        ## Keep these to default, because the fast versions of the environments implement different values ##
        ## of these variables for faster execution ##
        "vehicles_count": num_of_vehicles,
        "lanes_count": lane_count,
        "simulation_frequency": 5,
        "duration": 120,  # [s]
        "disable_collision_checks": True,    # Check collisions for other vehicles
        "enable_lane_change": False,
        ## Other parameters aleady set by default configurations ##
        # "action": {
        #     "type": "DiscreteMetaAction",
        # },
        # "controlled_vehicles": 1,
        # "initial_lane_id": None,
        # "ego_spacing": 2,
        # "vehicles_density": 1,
        # "collision_reward": -1,  # The reward received when colliding with a vehicle.
        # "right_lane_reward": 0.1,  # The reward received when driving on the right-most lanes, linearly mapped to
        # # zero for other lanes.
        # "high_speed_reward": 0.4,  # The reward received when driving at full speed, linearly mapped to zero for
        # # lower speeds according to config["reward_speed_range"].
        # "lane_change_reward": 0,  # The reward received at each lane change action.
        # "reward_speed_range": [20, 30],
        # "normalize_reward": True,
        # "offroad_terminal": False
        }

# env = gymnasium.make('highway-v0', config=config, render_mode='rgb_array')
# env = gymnasium.make('highway-fast-v0', config=config, render_mode='rgb_array')
env = gymnasium.make('highway-icy-custom-v0', config=config, render_mode='rgb_array')

_ = env.reset()
# action = env.unwrapped.action_type.actions_indexes["IDLE"]

In [3]:
import copy

In [5]:
env.unwrapped.road.vehicles[0].position

array([154.01276665,   8.        ])

In [33]:
env_copy = copy.deepcopy(env)
env_copy.unwrapped.road.vehicles[0].position = np.array([50.0, 4.0])
env_copy.unwrapped.road.vehicles

[MDPVehicle #200: [50.  4.],
 IDMVehicle #904: [179.50159323   0.        ],
 IDMVehicle #952: [204.93595799   4.        ]]

In [26]:
env_copy.step(0)
env_copy.unwrapped.road.vehicles

[MDPVehicle #344: [75.  4.],
 IDMVehicle #912: [200.95864985   0.        ],
 IDMVehicle #784: [226.00336446   4.82469402]]

In [26]:
# Create discrete world grid
temp = env.unwrapped.road.network.lanes_list()
x_max = max([lane.end[0] for lane in temp])
y_max = max([lane.end[1] for lane in temp])
x_grid = np.linspace(0, x_max, 500)
y_grid = np.linspace(0, y_max, lane_count)
[x_max, y_max]

[3000.0, 8.0]

In [3]:
%prun obs, reward, done, truncated, info = env.step(action)

         11827 function calls (11691 primitive calls) in 0.011 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      296    0.001    0.000    0.001    0.000 lane.py:209(local_coordinates)
      443    0.000    0.000    0.000    0.000 {method 'dot' of 'numpy.ndarray' objects}
      195    0.000    0.000    0.001    0.000 linalg.py:2383(norm)
      160    0.000    0.000    0.003    0.000 objects.py:133(_is_colliding)
       12    0.000    0.000    0.001    0.000 objects.py:180(polygon)
1173/1167    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       16    0.000    0.000    0.001    0.000 controller.py:145(steering_control)
      146    0.000    0.000    0.000    0.000 lane.py:80(on_lane)
      106    0.000    0.000    0.000    0.000 _methods.py:90(_clip)
        6    0.000    0.000    0.001    0.000 utils.py:194(are_polygons_intersecting)
       12    0.000    0.000    0.001    0.000 road.py:480(neig

In [4]:
env.unwrapped.road.vehicles

[MDPVehicle #824: [183.87107986   4.        ],
 IDMVehicle #216: [197.79668924   8.        ],
 IDMVehicle #904: [219.72090957   0.        ]]

In [5]:
env.unwrapped.road.objects

[Ice1 #760: at [151.090512   4.      ],
 Ice1 #432: at [218.85401083   8.        ],
 Ice1 #608: at [268.52539651   0.        ],
 Ice1 #528: at [294.43759937   0.        ],
 Ice1 #536: at [343.98100414   4.        ],
 Ice1 #712: at [392.53761495   8.        ],
 Ice1 #656: at [460.59978171   4.        ],
 Ice1 #528: at [505.44612845   0.        ],
 Ice1 #72: at [551.07811977   8.        ],
 Ice1 #64: at [633.43185766   0.        ]]

In [6]:
obs

array([[  1.      , 183.87108 ,   4.      ,  29.444445,   0.      ],
       [  1.      , 197.79669 ,   8.      ,  17.942114,   0.      ],
       [  1.      , 218.854   ,   8.      ,   0.      ,   0.      ],
       [  1.      , 219.72092 ,   0.      ,  16.230988,   0.      ],
       [  1.      , 268.5254  ,   0.      ,   0.      ,   0.      ],
       [  1.      , 294.4376  ,   0.      ,   0.      ,   0.      ],
       [  1.      , 343.98102 ,   4.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ]],
      dtype=float32)